<a href="https://colab.research.google.com/github/dcdlima/COS738/blob/main/COS738_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O dataset.zip https://github.com/LINE-PESC/Dados-Exercicios-Texto/raw/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
!unzip dataset.zip
!rm -rf dataset.zip
!rm -rf sample_data

!pip install nltk

--2025-07-24 11:50:51--  https://github.com/LINE-PESC/Dados-Exercicios-Texto/raw/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LINE-PESC/Dados-Exercicios-Texto/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip [following]
--2025-07-24 11:50:51--  https://raw.githubusercontent.com/LINE-PESC/Dados-Exercicios-Texto/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548260 (1.5M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip     

In [7]:
from pathlib import Path

configs_dir = Path('configs')
configs_dir.mkdir(exist_ok=True)

with open(configs_dir/'PC.CFG', 'w') as conf:
    conf.writelines('LEIA=data/cfquery.xml\n')
    conf.write('CONSULTAS=consultas.csv\n')
    conf.write('ESPERADOS=esperados.csv')

with open(configs_dir/'GLI.CFG', 'w') as conf:
    conf.write('LEIA=data/cf74.xml\n')
    conf.write('LEIA=data/cf75.xml\n')
    conf.write('LEIA=data/cf76.xml\n')
    conf.write('LEIA=data/cf77.xml\n')
    conf.write('LEIA=data/cf78.xml\n')
    conf.write('LEIA=data/cf79.xml\n')
    conf.write('ESCREVA=li.csv')

with open(configs_dir/'INDEX.CFG', 'w') as conf:
    conf.write('LEIA=li.csv\n')
    conf.write('ESCREVA=modelo_vetorial.json')

with open(configs_dir/'BUSCA.CFG', 'w') as conf:
    conf.write('MODELO=modelo_vetorial.json\n')
    conf.write('CONSULTAS=consultas.csv\n')
    conf.write('RESULTADOS=RESULTADOS.csv')

In [3]:
import xml.etree.ElementTree as ET
import csv
import unicodedata
import string
import logging
import time

# Função para normalizar texto
def normalizar_texto(texto):
    texto = texto.upper()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    texto = texto.translate(str.maketrans('', '', string.punctuation + ';'))
    return texto.strip()

# Iniciar log
logging.basicConfig(filename='processador_consultas.log', level=logging.INFO, format='%(asctime)s - %(message)s')
inicio_total = time.time()
logging.info("Início do Processador de Consultas")

# Ler arquivo de configuração PC.CFG
cfg_file = 'configs/PC.CFG'
with open(cfg_file, 'r', encoding='utf-8') as f:
    linhas_cfg = [linha.strip() for linha in f.readlines()]

arquivo_xml = linhas_cfg[0].split('=')[1]
arquivo_consultas = linhas_cfg[1].split('=')[1]
arquivo_esperados = linhas_cfg[2].split('=')[1]
logging.info(f"Arquivo de configuração lido: {cfg_file}")
logging.info(f"Arquivo XML: {arquivo_xml}, Consultas: {arquivo_consultas}, Esperados: {arquivo_esperados}")

# Ler e processar o arquivo XML
tree = ET.parse(arquivo_xml)
root = tree.getroot()
logging.info("Arquivo XML carregado")

# Gerar arquivo de consultas normalizadas
with open(arquivo_consultas, 'w', newline='', encoding='utf-8') as f_consultas:
    writer_consultas = csv.writer(f_consultas, delimiter=';')
    writer_consultas.writerow(['QueryNumber', 'QueryText'])

    for query in root.findall('QUERY'):
        query_number = query.find('QueryNumber').text.strip()
        query_text = query.find('QueryText').text.strip()
        query_text_normalizado = normalizar_texto(query_text)
        writer_consultas.writerow([query_number, query_text_normalizado])

logging.info("Arquivo de consultas normalizadas gerado")

# Gerar arquivo de resultados esperados
with open(arquivo_esperados, 'w', newline='', encoding='utf-8') as f_esperados:
    writer_esperados = csv.writer(f_esperados, delimiter=';')
    writer_esperados.writerow(['QueryNumber', 'DocNumber', 'DocVotes'])

    for query in root.findall('QUERY'):
        query_number = query.find('QueryNumber').text.strip()
        for item in query.find('Records').findall('Item'):
            score = int(item.attrib.get('Score', '0'))
            if score > 0:
                doc_number = item.text.strip()
                writer_esperados.writerow([query_number, doc_number, score])

logging.info("Arquivo de resultados esperados gerado")

fim_total = time.time()
logging.info(f"Processamento concluído em {fim_total - inicio_total:.2f} segundos")
print("Processador de Consultas executado com sucesso.")


import logging

# Configuração para salvar em arquivo E mostrar no terminal
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    handlers=[
        logging.FileHandler('processador_consultas.log', mode='w', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

!cat processador_consultas.log

Processador de Consultas executado com sucesso.


In [4]:
import xml.etree.ElementTree as ET
import csv
import unicodedata
import string
import logging
import time
import os
from collections import defaultdict

# Função para normalizar palavras
def normalizar_palavra(palavra):
    palavra = palavra.upper()
    palavra = unicodedata.normalize('NFKD', palavra).encode('ASCII', 'ignore').decode('ASCII')
    palavra = ''.join([c for c in palavra if c.isalpha()])
    return palavra if len(palavra) >= 1 else ''

# Iniciar log
logging.basicConfig(filename='gerador_lista_invertida.log', level=logging.INFO, format='%(asctime)s - %(message)s')
inicio_total = time.time()
logging.info("Início do Gerador de Lista Invertida")

# Ler arquivo de configuração GLI.CFG
cfg_file = 'configs/GLI.CFG'
with open(cfg_file, 'r', encoding='utf-8') as f:
    linhas_cfg = [linha.strip() for linha in f.readlines()]

arquivos_leia = []
arquivo_escreva = None
for linha in linhas_cfg:
    if linha.startswith('LEIA='):
        arquivos_leia.append(linha.split('=')[1])
    elif linha.startswith('ESCREVA='):
        arquivo_escreva = linha.split('=')[1]

logging.info(f"Arquivos XML a serem lidos: {arquivos_leia}")
logging.info(f"Arquivo de saída: {arquivo_escreva}")

# Dicionário para lista invertida
lista_invertida = defaultdict(list)

# Processar cada arquivo XML
for xml_file in arquivos_leia:
    if not os.path.exists(xml_file):
        logging.warning(f"Arquivo não encontrado: {xml_file}")
        continue

    tree = ET.parse(xml_file)
    root = tree.getroot()
    logging.info(f"Processando arquivo: {xml_file}")

    for record in root.findall('RECORD'):
        recordnum_elem = record.find('RECORDNUM')
        if recordnum_elem is None:
            continue
        doc_id = int(recordnum_elem.text.strip())

        abstract_elem = record.find('ABSTRACT')
        extract_elem = record.find('EXTRACT')
        texto = ''
        if abstract_elem is not None and abstract_elem.text:
            texto = abstract_elem.text
        elif extract_elem is not None and extract_elem.text:
            texto = extract_elem.text
        else:
            logging.warning(f"Documento {doc_id} sem ABSTRACT ou EXTRACT.")
            continue

        palavras = texto.split()
        for palavra in palavras:
            palavra_norm = normalizar_palavra(palavra)
            if palavra_norm:
                lista_invertida[palavra_norm].append(doc_id)

# Gerar arquivo CSV com lista invertida
with open(arquivo_escreva, 'w', newline='', encoding='utf-8') as f_out:
    writer = csv.writer(f_out, delimiter=';')
    for palavra, doc_ids in lista_invertida.items():
        writer.writerow([palavra, doc_ids])

fim_total = time.time()
logging.info(f"Processamento concluído em {fim_total - inicio_total:.2f} segundos")
print("Gerador de Lista Invertida executado com sucesso.")

Gerador de Lista Invertida executado com sucesso.


In [5]:
import csv
import math
import unicodedata
import string
import logging
import time
import pickle
from collections import defaultdict

# Função para normalizar palavras
def normalizar_palavra(palavra):
    palavra = palavra.upper()
    palavra = unicodedata.normalize('NFKD', palavra).encode('ASCII', 'ignore').decode('ASCII')
    palavra = ''.join([c for c in palavra if c.isalpha()])
    return palavra if len(palavra) >= 2 else ''

# Iniciar log
logging.basicConfig(filename='indexador.log', level=logging.INFO, format='%(asctime)s - %(message)s')
inicio_total = time.time()
logging.info("Início do Indexador")

# Ler arquivo de configuração INDEX.CFG
cfg_file = 'configs/INDEX.CFG'
with open(cfg_file, 'r', encoding='utf-8') as f:
    linhas_cfg = [linha.strip() for linha in f.readlines()]

arquivo_leia = None
arquivo_escreva = None
for linha in linhas_cfg:
    if linha.startswith('LEIA='):
        arquivo_leia = linha.split('=')[1]
    elif linha.startswith('ESCREVA='):
        arquivo_escreva = linha.split('=')[1]

logging.info(f"Arquivo de entrada: {arquivo_leia}")
logging.info(f"Arquivo de saída: {arquivo_escreva}")

# Estrutura para armazenar tf e df
tf = defaultdict(lambda: defaultdict(int))
df = defaultdict(int)
documentos = set()

# Ler lista invertida
with open(arquivo_leia, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    for linha in reader:
        if len(linha) != 2:
            continue
        termo = normalizar_palavra(linha[0])
        if not termo:
            continue
        try:
            lista_docs = eval(linha[1])
        except:
            continue
        df[termo] = len(set(lista_docs))
        for doc_id in lista_docs:
            tf[termo][doc_id] += 1
            documentos.add(doc_id)

N = len(documentos)
logging.info(f"Total de documentos: {N}")
logging.info(f"Total de termos distintos: {len(tf)}")

# Calcular tf-idf
modelo_vetorial = defaultdict(dict)
for termo, docs in tf.items():
    idf = math.log10(N / df[termo]) if df[termo] > 0 else 0
    for doc_id, freq in docs.items():
        tf_normalizado = 1 + math.log10(freq) if freq > 0 else 0
        modelo_vetorial[doc_id][termo] = tf_normalizado * idf

# Salvar modelo vetorial
with open(arquivo_escreva, 'wb') as f_out:
    pickle.dump(dict(modelo_vetorial), f_out)

fim_total = time.time()
logging.info(f"Indexação concluída em {fim_total - inicio_total:.2f} segundos")
print("Indexador executado com sucesso.")

Indexador executado com sucesso.


In [6]:
import csv
import pickle
import math
import logging
import time

# Função para calcular distância euclidiana
def distancia_euclidiana(vetor1, vetor2):
    termos = set(vetor1.keys()).union(set(vetor2.keys()))
    soma = 0
    for termo in termos:
        soma += (vetor1.get(termo, 0) - vetor2.get(termo, 0)) ** 2
    return math.sqrt(soma)

# Iniciar log
logging.basicConfig(filename='buscador.log', level=logging.INFO, format='%(asctime)s - %(message)s')
inicio_total = time.time()
logging.info("Início do Buscador")

# Ler arquivo de configuração BUSCA.CFG
cfg_file = 'configs/BUSCA.CFG'
with open(cfg_file, 'r', encoding='utf-8') as f:
    linhas_cfg = [linha.strip() for linha in f.readlines()]

arquivo_modelo = None
arquivo_consultas = None
arquivo_resultados = None
for linha in linhas_cfg:
    if linha.startswith('MODELO='):
        arquivo_modelo = linha.split('=')[1]
    elif linha.startswith('CONSULTAS='):
        arquivo_consultas = linha.split('=')[1]
    elif linha.startswith('RESULTADOS='):
        arquivo_resultados = linha.split('=')[1]

logging.info(f"Modelo: {arquivo_modelo}, Consultas: {arquivo_consultas}, Resultados: {arquivo_resultados}")

# Carregar modelo vetorial
with open(arquivo_modelo, 'rb') as f_modelo:
    modelo_vetorial = pickle.load(f_modelo)

# Ler consultas
consultas = []
with open(arquivo_consultas, 'r', encoding='utf-8') as f_consultas:
    reader = csv.reader(f_consultas, delimiter=';')
    next(reader)  # pular cabeçalho
    for linha in reader:
        if len(linha) != 2:
            continue
        query_id = linha[0]
        termos = linha[1].split()
        vetor_consulta = {termo: 1 for termo in termos}
        consultas.append((query_id, vetor_consulta))

logging.info(f"Total de consultas: {len(consultas)}")

# Realizar buscas
resultados = []
for query_id, vetor_consulta in consultas:
    ranking = []
    for doc_id, vetor_doc in modelo_vetorial.items():
        distancia = distancia_euclidiana(vetor_consulta, vetor_doc)
        ranking.append((doc_id, distancia))
    ranking.sort(key=lambda x: x[1])  # ordenar por distância
    lista_resultado = [(i+1, doc_id, round(distancia, 4)) for i, (doc_id, distancia) in enumerate(ranking)]
    resultados.append((query_id, lista_resultado))

# Salvar resultados
with open(arquivo_resultados, 'w', newline='', encoding='utf-8') as f_out:
    writer = csv.writer(f_out, delimiter=';')
    for query_id, lista in resultados:
        writer.writerow([query_id, lista])

fim_total = time.time()
logging.info(f"Busca concluída em {fim_total - inicio_total:.2f} segundos")
print("Buscador executado com sucesso.")



Buscador executado com sucesso.
cat: buscador.log: No such file or directory
